In [2]:
import os
import shutil
import re
import chardet
import pandas as pd
import requests
from bs4 import BeautifulSoup
from googlesearch import search
from unidecode import unidecode

In [3]:
data_dir = "C:/Users/Admin/Documents/NLP/text_mining/seminar/data"

food_urls = os.path.join(data_dir, "food_url.csv")
travel_urls = os.path.join(data_dir, "TRAVEL_url.csv")

In [4]:
def is_valid_url(url):
    try:
        response = requests.get(url, timeout=5)
        return response.status_code == 200  # True if URL is reachable
    except requests.RequestException:
        return False  # URL is broken or unreachable

In [5]:
# num_results = 100  # Number of links to fetch


def crawl_url(query, num_results):
    links = [url for url in search(query, num_results=num_results)]

    valid_links = [url for url in links if is_valid_url(url)]

    return valid_links

In [6]:
place = "TỈNH HẬU GIANG"
query = "du lịch " + place
hau_giang_20 = crawl_url(query, 20)

In [7]:
hau_giang_20_df = pd.DataFrame(
    {
        "STT": "20",
        "Tỉnh thành phố": "TỈNH HẬU GIANG",
        "Nguồn": hau_giang_20,
        "Title": None,
        "Description": None,
    }
)

In [8]:
# hau_giang_20_df.to_csv('a.csv')

In [9]:
hau_giang_20_df = hau_giang_20_df.rename(
    columns={"Tỉnh thành phố": "Location", "Nguồn": "Source"}
)
hau_giang_20_df["Location_norm"] = hau_giang_20_df["Location"].apply(
    lambda x: unidecode(x).lower().replace(" ", "_")
)
hau_giang_20_df["STT"] = hau_giang_20_df["STT"].astype(str)

In [10]:
hau_giang_20_df["fname"] = (
    hau_giang_20_df[["STT", "Location_norm"]].agg("_".join, axis=1)
    + "_"
    + hau_giang_20_df.index.astype(str)
    + ".txt"
)
hau_giang_20_df["fname"].head()

0    20_tinh_hau_giang_0.txt
1    20_tinh_hau_giang_1.txt
2    20_tinh_hau_giang_2.txt
3    20_tinh_hau_giang_3.txt
4    20_tinh_hau_giang_4.txt
Name: fname, dtype: object

In [11]:
travel_df = pd.read_csv(travel_urls)
food_df = pd.read_csv(food_urls)

In [12]:
def normalize_vietnamese_text(text):
    return unidecode(text)


# # Example usage
# vietnamese_text = "Chào mừng bạn đến với Hà Nội"
# normalized_text = normalize_vietnamese_text(vietnamese_text)
# print(normalized_text)  # Output: "Chao mung ban den voi Ha Noi"

In [13]:
travel_df

,STT,Tỉnh thành phố,Nguồn,Title,Description
0,1,THÀNH PHỐ HÀ NỘI,https://www.vietnamairlines.com/vn/vi/useful-i...,15++ Địa điểm du lịch Hà Nội hấp dẫn không thể...,Thủ đô ngàn năm văn hiến là điểm đến thu hút n...
1,1,THÀNH PHỐ HÀ NỘI,https://www.klook.com/vi/blog/dia-diem-du-lich...,30 Địa Điểm Du Lịch Hà Nội Nổi Tiếng Cập Nhật ...,Hà Nội có gì chơi? Bỏ túi ngay các địa điểm du...
2,1,THÀNH PHỐ HÀ NỘI,https://www.vntrip.vn/cam-nang/dia-diem-du-lic...,23 địa điểm tham quan du lịch ở Hà Nội hấp dẫn...,Hà Nội có rất nhiều địa điểm đẹp & nổi tiếng. ...
3,1,THÀNH PHỐ HÀ NỘI,https://vinwonders.com/vi/wonderpedia/news/top...,Địa điểm du lịch Hà Nội | TOP 31+ tọa độ ăn ch...,Địa điểm du lịch Hà Nội mang một nét đặc trưng...
4,1,THÀNH PHỐ HÀ NỘI,https://www.traveloka.com/vi-vn/explore/destin...,9 địa điểm du lịch ở Hà Nội chắc chắn bạn khôn...,Traveloka xin giới thiệu các địa điểm du lịch ...
...,...,...,...,...,...
625,64,TỈNH HẬU GIANG,https://cellphones.com.vn/sforum/dia-diem-du-l...,Top 15 địa điểm du lịch Hậu Giang nổi tiếng và...,Du lịch Hậu Giang có gì ăn chơi? Cùng Sforum k...
626,64,TỈNH HẬU GIANG,https://www.traveloka.com/vi-vn/explore/destin...,Bí quyết du lịch Hậu Giang từ A – Z: Đi đâu? Ă...,"Đến Hậu Giang đi đâu, ở khách sạn nào, ăn gì, ..."
627,64,TỈNH HẬU GIANG,https://haugiangtourism.com.vn/,Du lịch tỉnh Hậu Giang,Du lịch tỉnh Hậu Giang
628,64,TỈNH HẬU GIANG,https://vithuy.haugiang.gov.vn/xem-chi-tiet-ti...,Các điểm du lịch trên địa bàn huyện Vị Thủy,NaN


In [14]:
# Rename specific columns
travel_df = travel_df.rename(columns={"Tỉnh thành phố": "Location", "Nguồn": "Source"})
travel_df.columns

Index(['STT', 'Location', 'Source', 'Title', 'Description'], dtype='object')

In [15]:
travel_df["Location_norm"] = travel_df["Location"].apply(
    lambda x: unidecode(x).lower().replace(" ", "_")
)
travel_df["STT"] = travel_df["STT"].astype(str)

In [16]:
travel_df["fname"] = (
    travel_df[["STT", "Location_norm"]].agg("_".join, axis=1)
    + "_"
    + travel_df.index.astype(str)
    + ".txt"
)
travel_df["fname"].head()

0    1_thanh_pho_ha_noi_0.txt
1    1_thanh_pho_ha_noi_1.txt
2    1_thanh_pho_ha_noi_2.txt
3    1_thanh_pho_ha_noi_3.txt
4    1_thanh_pho_ha_noi_4.txt
Name: fname, dtype: object

In [17]:
# add urls of hau giang to raw dataframe

travel_df = pd.concat([travel_df, hau_giang_20_df], ignore_index=True)
# travel_df = travel_df.sort_values(by="STT")

In [18]:
def fetch_and_save(url, filename, output_dir="crawled_pages"):
    """Fetches a webpage and saves its text content to a file."""
    try:
        # Fetch the webpage
        response = requests.get(url, timeout=5)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)

        # Parse HTML content
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract the main text content
        text = soup.get_text(separator="\n", strip=True)

        # Generate a filename based on the URL
        # filename = url.replace("https://", "").replace("http://", "").replace("/", "_") + ".txt"
        filepath = os.path.join(output_dir, filename)

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Save content to file
        with open(filepath, "w", encoding="utf-8") as file:
            file.write(text)

        print(f"✅ Saved: {url} -> {filepath}")
        return filepath
    except requests.RequestException as e:
        print(f"❌ Failed to fetch {url}: {e}")
        return None

In [19]:
# for index, row in travel_df.iterrows():
#     # Access row values by column name
#     # print(index, row['STT'], row['fname'])
#     if is_valid_url(row["Source"]):
#         fetch_and_save(row["Source"], row["fname"], output_dir="travel")

In [20]:
travel_content_dir = "C:/Users/Admin/Documents/NLP/text_mining/seminar/travel"
even_dir = os.path.join(travel_content_dir, "even")
odd_dir = os.path.join(travel_content_dir, "odd")


if not os.path.exists(even_dir):
    os.makedirs(even_dir)
if not os.path.exists(odd_dir):
    os.makedirs(odd_dir)

In [21]:
for dirname, _, filenames in os.walk(travel_content_dir):
    for filename in filenames:
        src_path = os.path.join(dirname, filename)

        metadata = filename.split("_")
        order = int(metadata[0])

        if order % 2 == 1:
            des_path = os.path.join(odd_dir, filename)
        else:
            des_path = os.path.join(even_dir, filename)

        shutil.move(src_path, des_path)

In [22]:
travel_df

,STT,Location,Source,Title,Description,Location_norm,fname
0,1,THÀNH PHỐ HÀ NỘI,https://www.vietnamairlines.com/vn/vi/useful-i...,15++ Địa điểm du lịch Hà Nội hấp dẫn không thể...,Thủ đô ngàn năm văn hiến là điểm đến thu hút n...,thanh_pho_ha_noi,1_thanh_pho_ha_noi_0.txt
1,1,THÀNH PHỐ HÀ NỘI,https://www.klook.com/vi/blog/dia-diem-du-lich...,30 Địa Điểm Du Lịch Hà Nội Nổi Tiếng Cập Nhật ...,Hà Nội có gì chơi? Bỏ túi ngay các địa điểm du...,thanh_pho_ha_noi,1_thanh_pho_ha_noi_1.txt
2,1,THÀNH PHỐ HÀ NỘI,https://www.vntrip.vn/cam-nang/dia-diem-du-lic...,23 địa điểm tham quan du lịch ở Hà Nội hấp dẫn...,Hà Nội có rất nhiều địa điểm đẹp & nổi tiếng. ...,thanh_pho_ha_noi,1_thanh_pho_ha_noi_2.txt
3,1,THÀNH PHỐ HÀ NỘI,https://vinwonders.com/vi/wonderpedia/news/top...,Địa điểm du lịch Hà Nội | TOP 31+ tọa độ ăn ch...,Địa điểm du lịch Hà Nội mang một nét đặc trưng...,thanh_pho_ha_noi,1_thanh_pho_ha_noi_3.txt
4,1,THÀNH PHỐ HÀ NỘI,https://www.traveloka.com/vi-vn/explore/destin...,9 địa điểm du lịch ở Hà Nội chắc chắn bạn khôn...,Traveloka xin giới thiệu các địa điểm du lịch ...,thanh_pho_ha_noi,1_thanh_pho_ha_noi_4.txt
...,...,...,...,...,...,...,...
643,20,TỈNH HẬU GIANG,https://tuoitre.vn/de-nghi-hau-giang-xay-dung-...,None,None,tinh_hau_giang,20_tinh_hau_giang_13.txt
644,20,TỈNH HẬU GIANG,https://www.vntrip.vn/cam-nang/dia-diem-du-lic...,None,None,tinh_hau_giang,20_tinh_hau_giang_14.txt
645,20,TỈNH HẬU GIANG,https://www.ivivu.com/blog/2022/06/bo-tui-kinh...,None,None,tinh_hau_giang,20_tinh_hau_giang_15.txt
646,20,TỈNH HẬU GIANG,https://thamhiemmekong.com/thong-tin-du-lich-m...,None,None,tinh_hau_giang,20_tinh_hau_giang_16.txt


---

In [23]:
travel_df.iloc[629]["Source"]

'https://thuductravel.com.vn/12-dia-diem-tham-quan-du-lich-hau-giang-khong-the-bo-qua/'

In [24]:
# # add resource of content for each lines

# even_dir = "C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even"


# for dirname, _, filenames in os.walk(even_dir):
#     for filename in filenames:
#         src_path = os.path.join(dirname, filename)

#         metadata = filename.split(".")[0].split("_")
#         order = int(metadata[0])
#         idx = int(metadata[-1])

#         url = travel_df.iloc[idx]["Source"]

#         # with open(src_path,"a") as file:
#         #     file.write(f"{url}\n---\n")

#         print(src_path)
#         # print(filename,"\t: \t",url)

#         # if order%2 == 1:
#         #     des_path = os.path.join(odd_dir,filename)
#         # else:
#         #     des_path = os.path.join(even_dir,filename)

#         # shutil.move(src_path,des_path)

In [25]:
travel_df.to_csv("travel_full.csv")

In [31]:
def generate_latex_file(
    filename, title, abstract, place_sections=None, food_sections=None
):
    place_sections = place_sections or []  # Ensure it's a list
    food_sections = food_sections or []  # Ensure it's a list
    latex_template = f"""\\documentclass{{article}}
\\usepackage[vietnamese]{{babel}}
\\usepackage[letterpaper,top=1cm,bottom=1cm,left=1.5cm,right=1.5cm,marginparwidth=1.75cm]{{geometry}}
\\usepackage{{amsmath}}
\\usepackage{{graphicx}}
\\usepackage[colorlinks=true, allcolors=blue]{{hyperref}}
\\title{{{title}}}

\\begin{{document}}
\\begin{{center}}
    \\fontsize{{18}}{{20}}\\textbf{{{title}}}
\\end{{center}}
\\begin{{abstract}}
    {abstract}
\\end{{abstract}}
\\section*{{Điểm du lịch}}
"""

    for section in place_sections:
        latex_template += f"\\section{{{section['name']}}}\n{section['content']}\n\n"

    latex_template += "\\newpage\n\\section*{{Ẩm thực}}\n\\setcounter{section}{0}"

    for section in food_sections:
        latex_template += f"\\section{{{section['name']}}}\n{section['content']}\n\n"

    latex_template += "\n\\end{document}"

    with open(filename, "w", encoding="utf-8") as file:
        file.write(latex_template)

    print(f"LaTeX file '{filename}' has been generated.")


# # Example usage
# title = "Cẩm nang du lịch Hà Nội"
# abstract = "Những địa điểm du lịch Hà Nội luôn có sức hút rất lớn với các du khách..."
# place_sections = [
#     # {
#     #     "name": "Hồ Gươm",
#     #     "content": "Hồ Gươm là một địa điểm nổi tiếng nằm ở trung tâm Hà Nội.",
#     # },
#     # {
#     #     "name": "Văn Miếu - Quốc Tử Giám",
#     #     "content": "Đây là trường đại học đầu tiên của Việt Nam.",
#     # },
# ]

# food_sections = []  # Now an empty list instead of an empty dictionary

# generate_latex_file("hanoi_guide.tex", title, abstract, place_sections, food_sections)

In [27]:
name_mapping_df = travel_df[['Location','fname']]
name_mapping_df['pattern'] = name_mapping_df['fname'].apply(lambda x: re.search(r"_(.*)_", x).group(1) if re.search(r"_(.*)_", x) else None)
name_mapping_df['title'] = name_mapping_df['Location'].apply(lambda x: x.title())

title_df = name_mapping_df[['pattern','title']].drop_duplicates()
title_df

C:\Users\Admin\AppData\Local\Temp\ipykernel_26700\3826956386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_mapping_df['pattern'] = name_mapping_df['fname'].apply(lambda x: re.search(r"_(.*)_", x).group(1) if re.search(r"_(.*)_", x) else None)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26700\3826956386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_mapping_df['title'] = name_mapping_df['Location'].apply(lambda x: x.title())


,pattern,title
0,thanh_pho_ha_noi,Thành Phố Hà Nội
10,thanh_pho_ho_chi_minh,Thành Phố Hồ Chí Minh
20,thanh_pho_hai_phong,Thành Phố Hải Phòng
30,thanh_pho_da_nang,Thành Phố Đà Nẵng
40,tinh_ha_giang,Tỉnh Hà Giang
...,...,...
580,tinh_bac_lieu,Tỉnh Bạc Liêu
590,tinh_ca_mau,Tỉnh Cà Mau
600,tinh_dien_bien,Tỉnh Điện Biên
610,tinh_dak_nong,Tỉnh Đăk Nông


In [30]:
tex_path = "C:\\Users\\Admin\\Documents\\NLP\\text_mining\\seminar\\tex"

for index,row in title_df.iterrows():
    fname = row['pattern'] + ".tex"
    dest_path = os.path.join(tex_path,fname)
    title = f"Cẩm nang du lịch {row['title']}"
    abstract = "(Giới thiệu)"
    place_sections = []
    food_sections = []  # Now an empty list instead of an empty dictionary
    generate_latex_file(dest_path, title, abstract, place_sections, food_sections)

LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\thanh_pho_ha_noi.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\thanh_pho_ho_chi_minh.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\thanh_pho_hai_phong.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\thanh_pho_da_nang.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\tinh_ha_giang.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\tinh_cao_bang.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\tinh_lai_chau.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\tinh_lao_cai.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\text_mining\seminar\tex\tinh_tuyen_quang.tex' has been generated.
LaTeX file 'C:\Users\Admin\Documents\NLP\

In [ ]:
# ensure content is encoded in utf-8
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    return result["encoding"]


def convert_to_utf8(file_path):
    encoding = detect_encoding(file_path)
    if encoding is None:
        print(f"Skipping {file_path}: Encoding could not be detected.")
        return

    try:
        with open(file_path, "r", encoding=encoding) as f:
            content = f.read()
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"Converted {file_path} to UTF-8.")
    except Exception as e:
        print(f"Error converting {file_path}: {e}")


def convert_files_in_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                convert_to_utf8(file_path)


# Example usage
# directory_path = "path/to/your/directory"  # Change this to your directory path
convert_files_in_directory(even_dir)


Skipping C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_91.txt: Encoding could not be detected.
Skipping C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_92.txt: Encoding could not be detected.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_93.txt to UTF-8.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_95.txt to UTF-8.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_96.txt to UTF-8.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_97.txt to UTF-8.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_98.txt to UTF-8.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\10_tinh_lang_son_99.txt to UTF-8.
Converted C:/Users/Admin/Documents/NLP/text_mining/seminar/travel/even\12_tinh_thai_nguyen_110.txt to UTF-8.
Convert